# Milestone 2
Frederik Kallestrup Mastratis (qln174)  
Dongyu Liu (dlf327)  
Shamim Tariq Akram (zmx145)  
Celina Aurora Nguyen (szf345)  

Group 21

## Task 1 & 2

We have choosen the following schema for the tables in the database:

- A(id, domain, type, url, content, timestamps, title, summary)

- T(id:int, tagid:string)

- AU(a_id:int, author:string)

- K(a_id, keyword:string)

- MK(a_id, keyword:string)

Where italicization denotes that the field is (part of) the primary key.
With this design only the primary key is the superkey for the 
non-trivial functional dependencies. This also makes the schema trivially BCNF.

In [1]:
import pandas as pd
import numpy as np
import psycopg2

# Loading the first 10000 rows to compile faster
df_load = pd.read_csv("1mio-raw.csv", delimiter = ",", nrows = 10000)

In [2]:
# Splits a string into list
def string_splitter(string):
    lst = str(string).split(", ")
    filter_obj = filter(lambda x: x != "", lst)
    return list(filter_obj)

# Strip a string representation of list of strings
def string_stripper(string):
    lst = [i.strip() for i in string[1:-1].replace('\'',"").split(',')]
    filter_obj = filter(lambda x: x != "", lst)
    return list(filter_obj)

def string_filter(lst):
    filters = [lambda x: not x.isdigit(), lambda x: x != ""]
    filter_obj = filter(lambda x: all([f(x) for f in filters]), lst)
    return list(filter_obj)

df = df_load.copy()

# Dropping columns (setting new ID column later)
df = df.drop(columns = ['Unnamed: 0', 'id', 'source'])

# Set new ID column
df = df.rename_axis('id').reset_index()
df.set_index('id')

df = df.astype({'domain':str, 'type':str, 'url':str, 'content':str, 'scraped_at':str, 'inserted_at':str,
        'updated_at':str, 'title':str, 'authors':str, 'keywords':str, 'meta_keywords':str,
        'meta_description':str, 'tags':str, 'summary':str}, copy = False)

# Convert blank fields into NaN
#df = df.replace(r'^\s*$', np.nan, regex=True)

# Replace 'nan' strings with NaN
df = df.replace("nan", np.nan)

# Convert all strings into lower case:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

# Clean types
type_set = ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable','rumor']
df['type'] = df['type'].apply(lambda x: np.nan if x not in type_set else x)

# Clean timestamps
for column in ['scraped_at','inserted_at','updated_at']:
    df[column] = df[column].apply(lambda x: pd.to_datetime(x, errors='coerce'))

# Clean auhtors - separate into list of strings
df['authors'] = df['authors'].apply(lambda x: string_splitter(x) if pd.notnull(x) else x)

# Clean metakeywords - strip a string representation of list of strings
df['meta_keywords'] = df['meta_keywords'].apply(string_stripper)

# Clean tags
df['tags'] = df['tags'].apply(lambda x: string_splitter(x) if pd.notnull(x) else x)
df['tags'] = df['tags'].apply(lambda x: string_filter(x) if isinstance(x, list) else x)

# Replace NaN into empty lists
for column in ['authors', 'keywords', 'meta_keywords', 'tags']:
    df[column] = df[column].fillna("").apply(list)


In [3]:
# TEST CELL
df.to_csv('test.csv', index=False)
display(df)

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary
0,0,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,is life an illusion? researchers prove 'realit...,[sean martin],[],[],the universe ceases to exist when we are not l...,[],NaN
1,1,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,donald trump,"[linda rivera, conrad calvano, az gal, lincoln...",[],[],NaN,[],NaN
2,2,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,the los angeles police department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,donald trump,"[linda rivera, conrad calvano, az gal, lincoln...",[],[],NaN,[],NaN
3,3,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,the white house has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"more winning! israeli intelligence source, deb...","[cleavis nowell, cleavisnowell, clarence j. fe...",[],[],NaN,[],NaN
4,4,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“the time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“oh, trump, you coward, you just wait, we will...","[f.n. lehner, don spilman, clarence j. feinour...",[],[],NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,beforeitsnews.com,fake,http://beforeitsnews.com/tea-party/2017/12/a-c...,a christmas opportunity for unity\n\nheadline:...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,a christmas opportunity for unity,[freedom bunker],[],[],NaN,[],NaN
9996,9996,beforeitsnews.com,fake,http://beforeitsnews.com/tea-party/2017/12/our...,our christmas gift to you\n\nheadline: bitcoin...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,our christmas gift to you,[freedom bunker],[],[],NaN,[],NaN
9997,9997,canadafreepress.com,conspiracy,http://canadafreepress.com/print_friendly/a-le...,subscribe to canada free press for free\n\nthe...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,a letter to the senate,"[alan joel, because without america, there is ...",[],"[conservative news, conservative newspaper]","news, politics, editorials, commentary, canada...",[],NaN
9998,9998,canadafreepress.com,conspiracy,http://canadafreepress.com/print_friendly/obam...,obama has consistently talked about how he is ...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,obama’s policy proposals are the opposite of “...,"[alan joel, because without america, there is ...",[],"[conservative news, conservative newspaper]","news, politics, editorials, commentary, canada...",[],NaN


In [4]:
# Generates the tables of our database

tables = ["artikel", "author", "tags", "keywords", "metakeywords"]

create_tables_all = [
    """ 
    CREATE TABLE artikel (
        id INT, 
        domain VARCHAR, 
        type VARCHAR, 
        url VARCHAR, 
        content VARCHAR, 
        scraped_at TIMESTAMP, 
        inserted_at TIMESTAMP,
        updated_at TIMESTAMP, 
        title VARCHAR (256), 
        meta_description VARCHAR, 
        summary VARCHAR,

        PRIMARY KEY (id) 
    );
    """
    ,
    """
    CREATE TABLE author (
        a_id INT,
        author VARCHAR,
        PRIMARY KEY (a_id, author),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """ 
    CREATE TABLE tags (
        a_id INT, 
        tag VARCHAR, 
        PRIMARY KEY (a_id, tag),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """
    CREATE TABLE keywords (
        a_id INT, 
        keyword VARCHAR,
        PRIMARY KEY (a_id, keyword),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
    ,
    """
    CREATE TABLE metakeywords (
        a_id INT, 
        mkeyword VARCHAR,
        PRIMARY KEY (a_id, mkeyword),
        FOREIGN KEY (a_id)
            REFERENCES artikel (id)
            ON UPDATE CASCADE ON DELETE CASCADE
    );
    """
]

# FOR TAs please configure this with your own settings
conn = psycopg2.connect(dbname="fakenewsdb", user="postgres", password="1234")

cursor = conn.cursor()

for table in tables:
    cursor.execute("DROP TABLE IF EXISTS " + table + " CASCADE;")

for sql in create_tables_all:
    cursor.execute(sql)

conn.commit

<function connection.commit>

In [5]:
# Set of functions that puts the data from pd.dataframe into the right tables. Explodes the dataframe columns with
# list of strings into seperate entries each with their own key (a_id, string)

def projectrow2tuple(fields, row):
    return tuple(map(lambda f: row[f], fields))

def insertstring(table, n):
    return "INSERT INTO {} VALUES ({}) ON CONFLICT DO NOTHING".format(table, ", ".join(map(lambda _: '%s', range(n))))

def multi_insert(server, a_id, insert, xs):
    for x in xs:
        server.execute(insert, (a_id, x))
    

def insert_csv2rows(server, csv_row):
    A_domain = ['id', 'domain', 'type', 'url', 'content', 'scraped_at',
       'inserted_at', 'updated_at', 'title', 'meta_description', 'summary']
    AU_domain = ['id', 'authors']
    T_domain = ['id', 'tags']
    K_domain = ['id', 'keywords']
    MK_domain = ['id', 'meta_keywords']
    
    Atuple = projectrow2tuple(A_domain, csv_row)
    (a_id, tags) = projectrow2tuple(T_domain, csv_row)
    (_, au) = projectrow2tuple(AU_domain, csv_row)
    (_, kws) = projectrow2tuple(K_domain, csv_row)
    (_, mkws) = projectrow2tuple(MK_domain, csv_row)

    Ainsert = insertstring("artikel", len(Atuple))
    Tinsert = insertstring("tags", 2)
    AUinsert = insertstring("author", 2)
    Kinsert = insertstring("keywords", 2)
    MKinsert = insertstring("metakeywords", 2)

    server.execute(Ainsert, Atuple)
    insert = lambda ins, xs: multi_insert(server, a_id, ins, xs)
    insert(Tinsert, tags)
    insert(AUinsert, au)
    insert(Kinsert, kws)
    insert(MKinsert, mkws)


for _, row in df.iterrows():
    insert_csv2rows(cursor, row)

conn.commit

<function connection.commit>

## Task 3

Writing queries for Task 3

In [6]:
# QUERY 1

query1 = """
    SELECT distinct domain, type, scraped_at 
    FROM artikel
    WHERE type = 'reliable' AND scraped_at >= '2018-01-15'
"""

cursor.execute('rollback')
cursor.execute(query1)
cursor.fetchall()

[('christianpost.com',
  'reliable',
  datetime.datetime(2018, 1, 25, 16, 17, 44, 789555))]

In [7]:
# QUERY 2

query2 = """
    SELECT author
    FROM (
        SELECT COUNT(a_id) AS cnt, author 
        FROM author 
        WHERE a_id IN (
            SELECT id
            FROM artikel
            WHERE type = 'fake'
        ) 
        GROUP BY author
    ) AS x
    WHERE cnt = (SELECT MAX(cnt) FROM x)
"""

# This gives a max, but doesnt take into account if multiple authors write max number of fake articles
query2_pseudomax = """
    SELECT COUNT(a_id), author 
    FROM author 
    WHERE a_id IN (
        SELECT id
        FROM artikel
        WHERE type = 'fake'
    )
    GROUP BY author ORDER BY COUNT(a_id) DESC LIMIT 1
"""

cursor.execute('rollback')
cursor.execute(query2_pseudomax)
cursor.fetchall()

[(525, 'freedom bunker')]

In [8]:
# QUERY 3 - Need to join the metakeywords table on id and concatenate a single strings with keywords sorted in alphabetical order
# then we can use that to compare with other entries in the tables - Didnt have time to finish

query3 = """
    SELECT a_id, COUNT(*)
    FROM metakeywords
    GROUP BY a_id, mkeyword
    HAVING COUNT(*) > 1
"""

cursor.execute('rollback')
cursor.execute(query3)
cursor.fetchall()

[]

## Task 4

Writing queries for Task 4

In [9]:
# QUERY 1 - FINDING THE TOP 3 DOMAIN PUBLISHING THE MOST FAKE ARTICLES
q1 = """
    SELECT COUNT(id), domain
    FROM artikel
    WHERE id IN (
        SELECT id
        FROM artikel
        WHERE type = 'fake'
    )
    GROUP BY domain ORDER BY COUNT(id) DESC LIMIT 3
"""

cursor.execute('rollback')
cursor.execute(q1)
cursor.fetchall()

[(3615, 'beforeitsnews.com'),
 (22, 'thecommonsenseshow.com'),
 (18, 'conservativefighters.com')]

In [10]:
# QUERY 2 - WHICH TYPE OF ARTICLES IS DONALD TRUMP MOST OFTEN MENTIONED IN

q2 = """
    SELECT COUNT(id), type
    FROM (
      SELECT id, type, domain
      FROM artikel
      WHERE title LIKE '%trump%'
    ) as x
    GROUP BY type  ORDER BY COUNT(id) DESC LIMIT 3
"""

cursor.execute('rollback')
cursor.execute(q2)
cursor.fetchall()

[(254, 'fake'), (169, 'political'), (105, 'conspiracy')]

In [11]:
# QUERY 3 - No more time